<a href="https://colab.research.google.com/github/NatnaelT/DeepFish/blob/master/updatedShasta.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving data_in_out_cascade.csv to data_in_out_cascade.csv
User uploaded file "data_in_out_cascade.csv" with length 1521611 bytes


In [0]:
# Part 1 - Data Preprocessing

# Importing the libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

# Importing the training set

dataset_tran_test_val_in = pd.read_csv('profile_model_in_out.csv') #all inputs 2000 - 2017


dataset_tran_test_val_in = np.array(dataset_tran_test_val_in)

# Feature Scaling
from sklearn.preprocessing import MinMaxScaler
sc = MinMaxScaler(feature_range = (0, 1)) #For inputs


data_all_2000_2018_in_scaled = sc.fit_transform(dataset_tran_test_val_in)


## Structureing the training and testing data

# Predictors (we should have a predictor for the validation data set as well)
training_set_scaled = data_all_2000_2018_in_scaled[:5275,[1,2,4,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25]]

testing_set_scaled = data_all_2000_2018_in_scaled[5275:6207,[1,2,4,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25]]

validation_set_scaled = data_all_2000_2018_in_scaled[6207:-1,[1,2,4,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25]]

## Response (we should have responses only for training and testing data)
# Shasta
training_set_scaled_resp_shashta = data_all_2000_2018_in_scaled[1:5276,[26, 27,28]]

testing_set_scaled_resp_shashta = data_all_2000_2018_in_scaled[5276:6208,[26, 27,28]]

validation_set_scaled_resp_shashta = data_all_2000_2018_in_scaled[6208:,[26, 27,28]]

In [0]:

######################################## Training ############################

# Creating a data structure with different lag timesteps and 1 output
lag_day = 5

X_train = []
y_train = []
for i in range(lag_day, len(training_set_scaled)):
    X_train.append(training_set_scaled[i-lag_day:i, :]) #Assuming 5 days lag time affects, here discharge temperature not included  
    y_train.append(training_set_scaled_resp_shashta[i]) #Shashta

X_train, y_train = np.array(X_train), np.array(y_train)

# Reshaping
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 23))

#test_set_scaled = testing_set_scaled_resp_shashta

X_test = []
y_test = []
for i in range(lag_day, len(testing_set_scaled_resp_shashta)):
    X_test.append(testing_set_scaled[i-lag_day:i, :])
    y_test.append(testing_set_scaled_resp_shashta[i])


X_test, y_test = np.array(X_test), np.array(y_test)

# Reshaping for keras formating
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 23))


# Building the RNN

# Importing the Keras libraries and packages
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout

#from keras.layers import recurrent_dropout

# Initialising the RNN
model = Sequential()

# Adding the first LSTM layer and some Dropout regularisation
model.add(LSTM(units = 13, activation = 'sigmoid',return_sequences = True, input_shape = (X_train.shape[1], 23)))
model.add(Dropout(0.5))
# Adding a second LSTM layer and some Dropout regularisation
model.add(LSTM(units = 13 ,return_sequences = True))
model.add(Dropout(0.2))

model.add(LSTM(units = 3, activation = 'relu'))
model.add(Dropout(0.2))





#This part is forked from https://stackoverflow.com/questions/48924165/google-colaboratory-weight-download-export-saved-models
# Checkout https://github.com/MerceaOtniel/Google-Colaboratory-Tensorboard-Setup/blob/master/deep_learning_turkey.ipynb


!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# 1. Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# 2. Save Keras Model or weights on google drive

# create on Colab directory
#model.save('model.h5')  

model_SHA_P.save('shasta_profile.h5')

model_SHA_T.save('shasta_temp.h5')

model_KWK_T.save('keswick_temp.h5')


#model_file = drive.CreateFile({'title' : 'model.h5'})

model_SHA_P_file = drive.CreateFile({'title' : 'shasta_profile.h5'})

model_SHA_T_file = drive.CreateFile({'title' : 'shasta_temp.h5'})

model_KWK_T_file = drive.CreateFile({'title' : 'keswick_temp.h5'})



#model_file.SetContentFile('model.h5')

model_SHA_P_file.SetContentFile('shasta_profile.h5')

model_SHA_T_file.SetContentFile('shasta_temp.h5')

model_KWK_T_file.SetContentFile('keswick_temp.h5')


#model_file.Upload()


model_SHA_P_file.Upload()

model_SHA_T_file.Upload()

model_KWK_T_file.Upload()

# download to google drive
#drive.CreateFile({'id': model_file.get('id')})


drive.CreateFile({'id': model_SHA_P_file.get('id')})

drive.CreateFile({'id': model_SHA_T_file.get('id')})

drive.CreateFile({'id': model_KWK_T_file.get('id')})




























# Adding the output layer
model.add(Dense(units = 3))

# Compiling the RNN
model.compile(optimizer = 'adam', loss = 'mean_squared_error')


history = model.fit(X_train, y_train, epochs = 1500, batch_size = 128, validation_data = (X_test, y_test), verbose=2, shuffle=False)


## plot history
from matplotlib import pyplot
pyplot.plot(history.history['loss'], label='train')
pyplot.plot(history.history['val_loss'], label='test')
pyplot.legend()
pyplot.show()



X_valdidation = []

for i in range(lag_day, len(validation_set_scaled)):
    X_valdidation.append(validation_set_scaled[i-lag_day:i, :]) #predictors of the year 2017

X_valdidation = np.array(X_valdidation)

# Reshaping
X_valdidation = np.reshape(X_valdidation, (X_valdidation.shape[0], X_valdidation.shape[1], 23))


predicted_temp_scled = model.predict(X_valdidation)

sc1 = MinMaxScaler(feature_range = (0, 1)) #For inputs

data_all_2000_2018_in_scaled_out = sc1.fit_transform(dataset_tran_test_val_in[:,[26,27,28]])

data_observed = dataset_tran_test_val_in[6208:,[26, 27,28]]


predicted_temp = sc1.inverse_transform(predicted_temp_scled)


#testing_set_scaled_resp_shashta = data_all_2000_2018_in_scaled[5915:,[26, 27,28]]


# Visualising the results: Raw
plt.plot(data_observed[lag_day:,0], color = 'red', label = 'Actual_Middle ')
plt.plot(predicted_temp[:,0], color = 'blue', label = 'Predicted_Middle')
plt.title('Shasta Profile Temprature')
plt.xlabel('Time, # days')
plt.ylabel('Temperature, in oC')
plt.legend()
plt.show()

# Visualising the results: Raw
plt.plot(data_observed[lag_day:,1], color = 'red', label = 'Actual_Lower ')
plt.plot(predicted_temp[:,1], color = 'blue', label = 'Predicted_Lower')
plt.title('Shasta Profile Temprature')
plt.xlabel('Time, # days')
plt.ylabel('Temperature, in oC')
plt.legend()
plt.show()


# Visualising the results: Raw
plt.plot(data_observed[lag_day:,2], color = 'red', label = 'Actual_Side')
plt.plot(predicted_temp[:,2], color = 'blue', label = 'Predicted_Side')
plt.title('Shasta Profile Temprature')
plt.xlabel('Time, # days')
plt.ylabel('Temperature, in oC')
plt.legend()
plt.show()


#

from math import sqrt
from sklearn.metrics import mean_squared_error
rmse_middle = sqrt(mean_squared_error(data_observed[lag_day:,0], predicted_temp[:,0]))
rmse_low = sqrt(mean_squared_error(data_observed[lag_day:,1], predicted_temp[:,1]))
rmse_side = sqrt(mean_squared_error(data_observed[lag_day:,2], predicted_temp[:,2]))

print('Test RMSE: %.3f' % rmse_middle)
print('Test RMSE: %.3f' % rmse_low)
print('Test RMSE: %.3f' % rmse_side)